# Scraping: récupération des archives du journal Le Soir

## Imports

In [1]:
import os
import re
import time
import requests
from bs4 import BeautifulSoup

## Créer un répertoire pour stocker tous les articles

In [2]:
txt_path = '../data/archives_lesoir'
# Créer le dossier s'il n'existe pas
if not os.path.exists(txt_path):
    os.mkdir(txt_path)

## Récupérer les URLs de 100 articles contenant le mot "linguistique"

Le journal [Le Soir](https://www.lesoir.be) met à disposition un moteur de recherche pour accéder à ses archives depuis 1989 (2 millions d'articles).
Il est possible de faire des recherches ciblées et de récupérer les articles répondant à cette recherche.

Essayons une recherche sur le thème "linguistique". Le journal renvoie 10 articles par page. On va donc récupérer les articles sur les 10 premières pages.

In [3]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

articles_urls = list()
query = "linguistique"
for i in range(0, 10):
    print(f"Parsing page {i}")
    url = f"https://www.lesoir.be/archives/recherche?word={query}&sort=date%20desc&datefilter=anytime&start={i*10}"
    resp = requests.get(url, headers=headers)
    html = resp.text
    pattern = r"/art/[^\?]*"
    for art_url in re.findall(pattern, html):
        art_url = "https://www.lesoir.be" + art_url
        if art_url not in articles_urls:
            articles_urls.append(art_url)
    i += 10
    print(f"{len(articles_urls)} articles trouvés")


Parsing page 0
6 articles trouvés
Parsing page 1
10 articles trouvés
Parsing page 2
10 articles trouvés
Parsing page 3
10 articles trouvés
Parsing page 4
16 articles trouvés
Parsing page 5
20 articles trouvés
Parsing page 6
24 articles trouvés
Parsing page 7
28 articles trouvés
Parsing page 8
33 articles trouvés
Parsing page 9
33 articles trouvés


In [4]:
# Impression du nombre d'urls récupérées
print(len(articles_urls))
# Impression des 10 premières URLs
print(articles_urls[:10])


33
['https://www.lesoir.be/art/d-20220121-GT3E0E', 'https://www.lesoir.be/art/d-20220710-GW6N52', 'https://www.lesoir.be/art/d-20220829-GWZURP', 'https://www.lesoir.be/art/d-20220828-GWZ79R', 'https://www.lesoir.be/art/d-20220826-GWXYRY', 'https://www.lesoir.be/art/d-20220818-GWU1KZ', 'https://www.lesoir.be/art/d-20220811-GWPVM1', 'https://www.lesoir.be/art/d-20220804-GWL6RJ', 'https://www.lesoir.be/art/d-20220729-GWHFEQ', 'https://www.lesoir.be/art/d-20220728-GWGZ0L']


## Télécharger tous les articles et imprimer le titre, le chapô et la partie de l'article visible sans abonnement dans un fichier texte

In [11]:
for url in articles_urls:
    article_id = url.split("/")[-1]
    filename = os.path.join(txt_path, article_id + ".txt")
    if not os.path.exists(filename):
        
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.text, 'html.parser')
        title = soup.find("h1").get_text(separator=' ')
        chapo = soup.find("r-article--chapo").get_text(separator=' ')
        content = soup.find("r-article--section").get_text(separator=' ')
        
        with open(filename, 'w') as f:
            f.write(f"{title}\n\n{chapo}\n\n{content}")
    else:
        print(f"{article_id}: déjà téléchargé")
    

## Vérifier que tous les articles ont été téléchargés


Si ce n'est pas le cas, vous pouvez relancer l'étape de téléchargement (elle ignorera les documents déjà téléchargés)

In [14]:
ok_count = 0
for url in articles_urls:
    filename = url.split("/")[-1]
    downloads = os.listdir(txt_path)
    if f"{filename}.txt" not in downloads:
        print(f"{filename} is missing!")
    else:
        ok_count += 1
print(f"{ok_count} files found out of {len(articles_urls)}!")

33 files found out of 33!


## Pour en savoir plus

- Le web scraping avec Python: https://realpython.com/beautiful-soup-web-scraper-python/
- Tutoriel sur les expressions régulières: https://www.w3schools.com/python/python_regex.asp